In [ ]:
import RRQ.VQs as VQs
import RRQ.Tools as Tools
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sparse_ternary_networks.Tools import SubbandPCA_whitner
from scipy.linalg import toeplitz
from scipy.linalg import eigh

In [ ]:
N_test = 10000
N_train = 10000
n = 512
m = 512
numRep = 1
L = 250

In [ ]:
RhoList = [0.90]
#RhoList = [0,0.5,0.99]
#MethodList = ['RRQ']
MethodList = ['RQ']
for RhoInd,Rho in enumerate(RhoList):
    for Method in MethodList:
        exec('dist_' + Method + '_Rho' + str(int(Rho*100)) + '_train = np.zeros((numRep,L+1))')
        exec('dist_' + Method + '_Rho' + str(int(Rho*100)) + '_test = np.zeros((numRep,L+1))')
        #
        exec('rate_' + Method + '_Rho' + str(int(Rho*100)) + ' = np.zeros((numRep,L+1))')

In [ ]:
hparam_dict = {}
# Value = (lmda,gamma_p)
hparam_dict['RQ_Rho0'] = ([0]*L, 0.99)
hparam_dict['RQ_Rho50'] = ([0]*L, 0.99)
hparam_dict['RQ_Rho99'] = ([0]*L, 0.99)

hparam_dict['RRQ_Rho0'] = ([[np.inf for i in range(0,10)] + [np.inf]*1000][0] [:L], 0.0)
hparam_dict['RRQ_Rho50'] = ([[np.inf for i in range(-3,2)] + [np.inf]*1000][0] [:L], 0.0)
hparam_dict['RRQ_Rho99'] = ([[np.inf for i in range(-3,2)] + [np.inf]*1000][0] [:L], 0.0)


#hparam_dict['RRQ_Rho0'] = ([1e4,1e5,1e6,1e7,1e8,1e9,1e9][:L], 0.99)
#hparam_dict['RRQ_Rho50'] = ([1e-1,1e0,1e1,1e2,1e3,1e4,1e5][:L], 0.99)
#hparam_dict['RRQ_Rho99'] = ([1e-3,1e-2,1e-1,1e0,1e1,1e2,1e3][:L], 0.99)

In [ ]:
# ind = 1000
# plt.plot(F_train[:,ind])
# #plt.plot(F_hat[0][:,ind]  )
# plt.plot( F_hat[1][:,ind]  )

# plt.plot(obj_train.C[1][:,10])
#obj_train.init_method

In [ ]:
for RepInd in range(numRep):
    print('.......................................  Repetition = ',RepInd + 1)
    for RhoInd,Rho in enumerate(RhoList):
        print('....................  Rho = ',Rho)
        F_train,F_test = Tools.data_generator(n,N_train,N_test,'AR1',SourceParam=Rho)
        norm_train = np.divide(np.linalg.norm(F_train) ** 2, np.prod(F_train.shape))
        norm_test = np.divide(np.linalg.norm(F_test) ** 2, np.prod(F_test.shape))
        for MethodInd,Method in enumerate(MethodList):
            print(' ############   ' + Method + '     A L G O R I T H M   ################')
            if Method == 'RRQ':
                ######### Whitening:
                F_train, EigVecs, dim_means = SubbandPCA_whitner(F_train, numSB=1, EigVecs=None,dim_means=None)
                F_test, _, _ = SubbandPCA_whitner(F_test, numSB=1, EigVecs=EigVecs, dim_means=dim_means)
                ###############################################
            # training:
            exec("lmda = hparam_dict['"+ Method +"_Rho"+str(int(Rho*100))+"'][0]")
            exec("gamma_p = hparam_dict['"+ Method +"_Rho"+str(int(Rho*100))+"'][1]")
            obj_train = VQs.RRQ(L, m, lmda=lmda, init_method='Data', num_iter=(10, 8), eta=5e-1, gamma_p=gamma_p)
            F_out, F_hat, X_ind = obj_train.run(F_train)
            exec('dist_' + Method + '_Rho' + str(int(Rho*100)) + '_train[RepInd,:] = obj_train.distortion/norm_train')
            exec('rate_' + Method + '_Rho' + str(int(Rho*100)) + '[RepInd,:] = obj_train.rate')
            # testing:
            obj_test = VQs.VQ(obj_train.C)
            F_out, _, _ = obj_test.run(F_test)
            exec('dist_' + Method + '_Rho' + str(int(Rho*100)) + '_test[RepInd,:] = obj_test.distortion/norm_test')


In [ ]:
#plt.plot(np.mean(dist_RQ_Rho0_test,axis=0))
#plt.plot(np.mean(dist_RRQ_Rho0_test,axis=0))

In [ ]:
#plt.plot(np.mean(dist_RQ1_exp2_test,axis=0))
#plt.plot(np.mean(dist_RRQ_exp2_test,axis=0))
curve_dict = {}
for RhoInd,Rho in enumerate(RhoList):
    for MethodInd,Method in enumerate(MethodList):
        exec('x = np.mean(rate_' +  Method + '_Rho' + str(int(Rho*100)) + ',axis=0)')
        exec('y = np.mean(dist_' +  Method + '_Rho' + str(int(Rho*100)) + '_train,axis=0)')
        exec("curve_dict['Rho" + str(int(Rho*100))+ '_n' + str(n) + '_m' + str(m) +"_DR_"+ Method + "_train'] = (x,y)")
        
        exec('y = np.mean(dist_' +  Method + '_Rho' + str(int(Rho*100)) + '_test,axis=0)')
        exec("curve_dict['Rho" + str(int(Rho*100))+ '_n' + str(n)+ '_m' + str(m) +"_DR_"+ Method + "_test'] = (x,y)") 

In [ ]:
# # SLB calculation for AR(1) sources:
# Rate_SLB= np.linspace(0,4,1000)
# Distortion_SLB = np.zeros((len(RhoList),len(Rate_SLB)))
# for RhoInd,Rho in enumerate(RhoList):
#     print('Rho = ',Rho)
#     if Rho == 0:
#         Distortion_SLB[RhoInd,:] = 2**(-2*Rate_SLB)
#         continue
#     cov_mat = toeplitz(Rho ** np.arange(n))
#     EigValues, _ = eigh(cov_mat)
#     for RateInd,Rate in enumerate(Rate_SLB):
#         Distortion_SLB[RhoInd,RateInd] = np.sum(Tools.rev_WaterFiller(EigValues,Rate)[3])/n
        
        
# #
# plt.subplot(1,2,1)
# plt.plot(Rate_SLB,Distortion_SLB[RhoInd,:],color='BLACK') 
# plt.grid()
# plt.xlabel('Rate')
# plt.ylabel('Distortion')
# #
# plt.subplot(1,2,2)
# plt.semilogy(Rate_SLB,Distortion_SLB[RhoInd,:],color='BLACK') 
# plt.grid()
# plt.xlabel('Rate')
# plt.ylabel('Distortion')



# curve_dict['Rho0_SLB'] = (Rate_SLB,Distortion_SLB[0,:])
# curve_dict['Rho50_SLB'] = (Rate_SLB,Distortion_SLB[1,:])
# curve_dict['Rho991_SLB'] = (Rate_SLB,Distortion_SLB[2,:])

In [ ]:
# Saving the results for PGFplott:
PGF_path = 'PGF/dat/'
ExpName = 'SyntheticAR1_RRQ'
#
for curve_name, curve_xy in curve_dict.items():
    fname = ExpName + '_' + curve_name + '.dat'
    print(fname)
    x = curve_xy[0].astype(float).reshape(-1)
    y = curve_xy[1].astype(float).reshape(-1)
    np.savetxt(PGF_path + fname, np.transpose([x,y]), fmt='%8f', delimiter='   ')